# 11. 9号新数据（与BMC Genomics相同）

## 大概流程
+ 数据清洗（FillNA）
+ 特征方差过滤（特征工程）
    + 过滤后的特征一共51维（原始86维，方差阈值取1）
+ <font color='red'>New</font> 用贝叶斯优化来调参（取代GridSearchCV）
    + 参考 https://github.com/fmfn/BayesianOptimization
    + 保留最优参数组合
+ Cross Validation with Over-sampling
    + Plotting
    
## 细节-方差过滤

In [ ]:
from FuncLoadData import LoadData
import numpy as np

DataFilePath = 'BMCRawData/Data.csv'
PositivePath = 'BMCRawData/Positive.csv'
NegativePath = 'BMCRawData/Negative.csv'

ShuffleSeed = 442
X, y, Valid = LoadData(DataFilePath, PositivePath, NegativePath, ShuffleSeed)

X = X.fillna(0)
# X = np.asarray(X)

Valid = Valid.fillna(0)
# Valid = np.asarray(Valid)

FeatureName = X.columns.values
# print (FeatureName)

from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold = 1)
selector.fit(X)
RemainedFeatureLoc = selector.get_support(indices=True)
# np.savetxt("VarianceFilteredLoc.txt", RemainedFeatureLoc, fmt='%d')
X = selector.transform(X)
Valid = selector.transform(Valid)

# %%
RemainedFeatureName = FeatureName[RemainedFeatureLoc]
import pandas as pd

TrainingSet = pd.DataFrame(X, index=None, columns=RemainedFeatureName)
TrainingSet.to_csv('BMCProcessedData/TrainingSet.csv', index=None)

ValidSet = pd.DataFrame(Valid, index=None, columns=RemainedFeatureName)
ValidSet.to_csv('BMCProcessedData/ValidSet.csv', index=None)

np.savetxt('BMCPProcessedData/TrainingLabel.txt', y, fmt='%d')

## 细节-贝叶斯优化

这部分需要一个新的package，参考那个reference。 安装方法（命令行）：

In [ ]:
pip install bayesian-optimization

安装好以后，定义一个function，用来返回一个estimator的cross validation mean score（<font color='red'>xgb_cv</font>）  
然后定义一个贝叶斯优化器（<font color='red'>optimize_xgb</font>），定义一个参数的选取范围，求取使上一个function取得最大值的参数。  
完整过程如下：

In [ ]:
import numpy as np
import pandas as pd
from bayes_opt import BayesianOptimization
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import warnings

# Prepare xgb for bayes opt
def xgb_cv(n_estimator, max_depth, learning_rate, col_bytree, gamma, subsample, data, targets):
    estimator = XGBClassifier(n_estimators=n_estimator, 
                    max_depth=max_depth, 
                    learning_rate=learning_rate, 
                    colsample_bytree=col_bytree, 
                    gamma=gamma, 
                    subsample=subsample)
    cval = cross_val_score(estimator, data, targets, scoring='roc_auc', cv=10)
    return cval.mean()

def optimize_xgb(data, targets):
    """Apply Bayesian Optimization to Xgb parameters."""
    def xgb_crossval(n_estimators, max_depth, learning_rate, colsample_bytree, gamma, subsample):
        return xgb_cv(n_estimator = int(n_estimators), 
                    max_depth = int(max_depth), 
                    learning_rate = learning_rate, 
                    col_bytree = colsample_bytree,
                    gamma = gamma, 
                    subsample = subsample,
                    data=data, 
                    targets=targets)

    optimizer = BayesianOptimization(
        f=xgb_crossval,
        pbounds={'n_estimators': (10, 2000),
            'max_depth': (3, 10),
            'learning_rate': (0.01, 0.3),
            'colsample_bytree': (0.7, 1),
            'gamma': (0, 0.05),
            'subsample': (0.7, 1)},
        random_state=442)
#     Iteration for 20 times
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        optimizer.maximize(n_iter=20, acq='ei')

    return optimizer

if __name__ == "__main__":
    # Load Data
    X = pd.read_csv("BMCProcessedData/TrainingSet.csv")
    y = np.loadtxt("BMCProcessedData/TrainingLabel.txt", dtype = int)

    # Over-sampling
    from imblearn.over_sampling import SMOTE
    resampler = SMOTE(kind='svm', random_state=442)
    X_res, y_res = resampler.fit_resample(X, y)

    # Standardizing
    X_res = StandardScaler().fit_transform(X_res)

    OptRes = optimize_xgb(X_res, y_res)
    print("Final result:", OptRes.max)

    history_df = pd.DataFrame(OptRes.res)
    history_df.to_csv('Porto-AUC-10fold-XGB.csv')

最终结果如下：
+ colsample_bytree = 0.94
+ gamma = 0.03
+ learning_rate = 0.124
+ max_depth = 10
+ n_estimators = 1998
+ subsample=0.718

## 细节-带Over-sampling的交叉验证

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from xgboost.sklearn import XGBClassifier
from scipy import interp
from sklearn.metrics import roc_curve, auc, average_precision_score, precision_recall_curve
from matplotlib import pyplot as plt

def GetData():
    X = pd.read_csv("BMCProcessedData/TrainingSet.csv")
    y = np.loadtxt("BMCProcessedData/Traininglabel.txt", dtype=int)

    return np.asarray(X), y

def GetFoldResult(data, targets, cv, seed):
    Stratified_folder = StratifiedKFold(n_splits=cv, random_state=seed)
    FolderRes = Stratified_folder.split(data, targets)

    data = StandardScaler().fit_transform(data)

    # Get each k-fold results
    FoldXtrain = []
    FoldXtest = []
    Foldytrain = []
    Foldytest = []

    for train_index, test_index in FolderRes:
        # Original result for each fold
        X_train = data[train_index, :]
        y_train = targets[train_index]

        X_test = data[test_index, :]
        y_test = targets[test_index]
        # For each fold, resample the training one
        resampler = SMOTE(kind='svm', random_state=seed)
        X_res, y_res = resampler.fit_resample(X_train, y_train)

        FoldXtrain.append(X_res)
        Foldytrain.append(y_res)
        FoldXtest.append(X_test)
        Foldytest.append(y_test)

    return FoldXtrain, FoldXtest, Foldytrain, Foldytest

def GetCVScore(X_trainList, X_testList, y_trainList, y_testList, estimator, cv):
    print ("=======================\nStart calculating AUC for each fold:")
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    tprlist = []
    fprlist = []

    precisionlist = []
    recalllist = []
    pres = []
    yreal = []
    yprob = []
    cv_pr = []

    for i in range(cv):
        X_train = X_trainList[i]
        y_train = y_trainList[i]

        X_test = X_testList[i]
        y_test = y_testList[i]

        clf = estimator
        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)
        y_pred_prob = clf.predict_proba(X_test)[:, 1]

        # ROC 
        fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
        fprlist.append(fpr)
        tprlist.append(tpr)
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)

        # PRC
        _precision, _recall, _ = precision_recall_curve(y_test, y_pred_prob)
        precisionlist.append(_precision)
        recalllist.append(_recall)
        pres.append(interp(mean_fpr, _recall[::-1], _precision[::-1]))
        cv_pr.append(average_precision_score(y_test, y_pred_prob))
        yreal.append(y_test)
        yprob.append(y_pred_prob)

        print ("Fold {} done".format(i+1))
    yreal = np.concatenate(yreal)
    yprob = np.concatenate(yprob)
    print ("=======================")
    return fprlist, tprlist, tprs, aucs, pres, yreal, yprob, cv_pr, precisionlist, recalllist
    
def PlotROC(fprlist, tprlist, tprs, aucs, cvfold):
    print ("Ploting!")
    mean_fpr = np.linspace(0, 1, 100)
    mean_tpr = np.mean(tprs, axis = 0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)

    plt.figure(figsize=(8, 7))
    # Plot each fold
    for i in range(cvfold):
        plt.plot(fprlist[i], 
                tprlist[i], 
                alpha=0.3, 
                label = "ROC fold %d (AUC = %.2f)"%(i, aucs[i]))
    # Plot the chance
    plt.plot([0, 1], [0, 1], linestyle='--', color='r', alpha=.8)
    # Plot the mean one
    plt.plot(mean_fpr, 
            mean_tpr, 
            color='b', 
            label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc), alpha=.8)
    # Plot the variances
    plt.fill_between(mean_fpr, 
                    tprs_lower, 
                    tprs_upper, 
                    color='grey', 
                    alpha=.2, label=r'$\pm$ 1 std. dev.')

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    # plt.title('ROC for each fold')
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.)
    plt.tight_layout()

    return plt


def PlotPRC(pres, yreal, yprob, prscores, prelist, reclist, cvfold):
    print ("Ploting!")
    mean_rec = np.linspace(0, 1, 100)
    mean_pre = np.mean(pres, axis = 0)
    mean_pre2, mean_rec2, _ = precision_recall_curve(yreal, yprob)
    mean_score = average_precision_score(yreal, yprob)

    std_score = np.std(prscores)

    std_pre = np.std(pres, axis=0)
    pres_upper = np.minimum(mean_pre + std_pre, 1)
    pres_lower = np.maximum(mean_pre - std_pre, 0)

    plt.figure(figsize=(8, 7))
    # Plot each fold
    for i in range(cvfold):
        plt.plot(reclist[i], 
                prelist[i],
                alpha=0.3, 
                label = "PRC fold %d (AP = %.2f)"%(i, prscores[i]))
    # Plot the mean one
    plt.plot(mean_rec2, 
            mean_pre2, 
            color='b', 
            label=r'Mean PRC (AP = %0.2f $\pm$ %0.2f)' % (mean_score, std_score), alpha=.8)
    # Plot the variances
    plt.fill_between(mean_rec, 
                    pres_lower, 
                    pres_upper, 
                    color='grey', 
                    alpha=.2, label=r'$\pm$ 1 std. dev.')

    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('Recall', fontsize=16)
    plt.ylabel('Precision', fontsize=16)
    # plt.title('ROC for each fold')
    plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=2, mode="expand", borderaxespad=0.)
    plt.tight_layout()

    return plt


if __name__ == "__main__":
    cv = 10
    # seed = 42435 42435 makes auc 0.84
    seed = 42435

    X, y = GetData()
    X_trainList, X_testList, y_trainList, y_testList = GetFoldResult(X, y, cv, seed)

    estimator = XGBClassifier(
            objective='binary:logistic', 
            colsample_bytree = 0.94, 
            gamma = 0.03, 
            learning_rate = 0.124, 
            max_depth = 10, 
            n_estimators = 1998, 
            subsample=0.718
        )

    fprlist, tprlist, tprs, aucs, pres, yreal, yprob, prscores, prelist, reclist = GetCVScore(X_trainList, X_testList, y_trainList, y_testList, estimator, cv)

    PlotROC(fprlist, tprlist, tprs, aucs, cv)
#     plt.savefig("10FoldCV_Resample_ROC.png",bbox_inches = "tight")
#     plt.savefig("10FoldCV_Resample_ROC.eps",bbox_inches = "tight")

    PlotPRC(pres, yreal, yprob, prscores, prelist, reclist, cv)
#     plt.savefig("10FoldCV_Resample_PRC.png",bbox_inches = "tight")
#     plt.savefig("10FoldCV_Resample_PRC.eps",bbox_inches = "tight")
    plt.show()

这里附上Plotting的结果：  
ROC:  
![title](Imgs/10FoldCV_Resample_ROC.png)  
PRC:  
![title](Imgs/10FoldCV_Resample_PRC.png)  

Cross Validation Table:  

|      x   |Precision | Recall | F1-Score | Support |
|------------|------|------|------|-----|
| macro avg fold 0 | 0.74 | 0.56 | 0.59 | 472 |
| macro avg fold 1 | 0.79 | 0.59 | 0.63 | 472 |
| macro avg fold 2 | 0.60 | 0.58 | 0.59 | 472 |
| macro avg fold 3 | 0.82 | 0.68 | 0.73 | 471 |
| macro avg fold 4 | 0.67 | 0.59 | 0.62 | 471 |
| macro avg fold 5 | 0.65 | 0.65 | 0.65 | 471 |
| macro avg fold 6 | 0.69 | 0.62 | 0.64 | 471 |
| macro avg fold 7 | 0.76 | 0.68 | 0.71 | 471 |
| macro avg fold 8 | 0.67 | 0.59 | 0.62 | 470 |
| macro avg fold 9 | 0.74 | 0.60 | 0.63 | 470 |

这里的support是只测试集的样本数量（每一个fold的测试集，1/10，差不多就是470个）。 总体的平均值和AUC/PR的值我没有加上去  
使用macro avg是因为我们更关注分类器在少数量类别上的表现。


## 下面我要做的

我想在特征工程上再做些东西，毕竟我们现在只用了个方差过滤, 而目前的性能差距： 只方差过滤(84%)$\approx$方差过滤后再特征权重过滤(84%)$\gg$不做特征工程(82%)

我目前的设想是这样的：
+ 用某种方式做特征的表示的参考标准， 比如， 用信息粒度（Info）
+ 然后用某种相似度方式找出这个标准下相似的特征，想办法合并,得到一个新的人工合成特征，起到一个特征工程/降维的作用
+ 大概表示如下：
    + Synthetized Feature = Integrate(Similarity(InfoA, InfoB, ..., InfoN))
+ 这个需要点时间，估计要年后了

另外，单边分类(One Class Classification)，以及各种异常检测的方法都可以用在这里（这里少数类就可以当成异常）